This notebook goes through a simple binary classification example, explaining general library functionality and data loading along the way.

We perform binary classification on sex, using structural MRI rois.

# Init Phase

In [ ]:
from ABCD_ML.ABCD_ML import ABCD_ML

Replace these values below with your own directory / desired files.

In [ ]:
#Define directory with the 2.0_NDA_Data
nda_dr = '/mnt/sdb2/2.0_ABCD_Data_Explorer/2.0_NDA_Data/'

#This file stores the name mapping
test_mapping_loc = nda_dr + 'ABCD_Release_ Notes_Data_Release_ 2.0/22. ABCD_Release_2.0_mapping_r.csv'

#We will use as the neuroimaging data just the sMRI data
test_data_loc1 = nda_dr + 'MRI/ABCD sMRI Part 1.csv'
test_data_loc2 = nda_dr + 'MRI/ABCD sMRI Part 2.csv'

#We will load target data (and covariate data) from here
test_target_loc = nda_dr + 'Mental Health/ABCD Parent Demographics Survey.csv'

#We will load stratification data from here
test_strat_loc = nda_dr + 'Other Non-Imaging/ABCD ACS Post Stratification Weights.csv'

#We will load exclusions from here, it is the list of flipped subject ids
test_exclusion_loc = '/home/sage/bader_things/invalid_pguids.txt'

We first need to define the class object, which we will use to load load and to train/test different ML models.
There are a few global parameters which we can optionally set when defining this object as well, lets look and see what they are.

In [ ]:
#help(ABCD_ML.__init__)

Most of the default parameters are okay for this simple example, but any of them can be changed.

One thing we want to change just to make things easier is setting the default_dataset_type field, as in this example all of the datasets we are loading from are 'explorer' type. This way we won't have to pass that to every loader.

In [ ]:
ML = ABCD_ML(default_dataset_type='explorer', existing_log='overwrite')

# Data Loading Phase

We can continue by optionally loading in a name map, which is simply a dictionary that attempts to rename any column names loaded in, if those column names are a key in the dictionary. This is useful for ABCD data as the default column names might not be useful.

Note this name map and these parameters are for the 'ABCD 2.0 Explorer' formatting. 

In [ ]:
ML.Load_Name_Map(loc = test_mapping_loc,
                 source_name_col = "NDAR name",
                 target_name_col = "REDCap name/NDA alias")

We can look at what exactly is in this dictionary if we want to confirm we loaded it correctly.
It is loaded as name_map within the ABCD_ML class.

In [ ]:
#some_examples = {k: ML.name_map[k] for k in list(ML.name_map)[300:320]}
#some_examples

In [ ]:
#help(ML.Load_Exclusions)

For our exclusions we will just use the flipped subjects

In [ ]:
ML.Load_Exclusions(loc=test_exclusion_loc)

Next, we will load in the actual data. Like before we can check what parameters this function wants / can accept

In [ ]:
#help(ML.Load_Data)

In [ ]:
#ML.Load_Data(loc=[test_data_loc1, test_data_loc2],
#             dataset_type='explorer',
#             filter_outlier_percent=.005, 
#             winsorize_val=.01)           

That ends up being a lot of data dropped just for dropping missing outliers... since we are not in low_memory_mode, we can just clear the data, and reload it. This time we will also load not just the first data loc, but the rest as well - and at the same time - but just providing the locations of both in a list.

In [ ]:
#ML.Clear_Data()

In [ ]:
ML.Load_Data(loc=[test_data_loc1, test_data_loc2],
             dataset_type='explorer', #This is set as default, but how we would specify it if we wanted to change i
             #filter_outlier_percent=.0005,
             filter_outlier_percent=.01,
             winsorize_val=.001)   

These seem okay settings.

Our data for this expiriment should now be loaded. We can check it out:

In [ ]:
#ML.data.head()

In [ ]:
#ML.data.shape

Great. Now that data is loaded we still need to load targets, and can optionally load covars, strat and exclusions. Lets load our target first, and begin as before by checking out the loading function.

In [ ]:
#help(ML.Load_Targets)

Great, lets just load in sex as our target variable.

In [ ]:
ML.Load_Targets(loc=test_target_loc,
                col_name='demo_sex_v2',
                data_type='b')

In [ ]:
ML.Show_Targets_Dist()

If you read the verbose print out above, you'll notice that it says "More than two unique score values found,filtered all but [1. 2.]" This is because by default when a binary datatype is passed, the dataloader needs to make sure it loads in only two unique values. To solve this when there exists outliers, like in this case, all but the top two unique values by count will be dropped. It will further show which values it has kept, in the case that an error was made, but here 1 and 2 are the correct sex values. If more than two values are desired, the categorical data type should be used.

Let's look and see to make sure everything was loaded correctly.

In [ ]:
#ML.targets.head()

We can look into adding covars next. Where co-variates arn't quite treated as typical co-variates, but are values we would like to be able to pass as additional input to the ML model if desired (and input that is treated in a special way, specifically covar input won't be scaled with any data scaler by default). 

In [ ]:
#help(ML.Load_Covars)

In [ ]:
ML.Load_Covars(loc=test_target_loc,
               col_names = 'demo_ed_v2',
               data_types = 'o',
               standardize = False,
               normalize = False) 

And check to see it was loaded correctly (and normalized)

In [ ]:
#ML.covars.head()

Lastly, we will considering loading different stratification values. These are the values that we can optionally define custom validation / split behavior on. Within this example, we are just going to make sure that all splits preserve subjects with the same family id within the same fold, so lets load family id - after looking as the help function.

In [ ]:
#help(ML.Load_Strat)

In [ ]:
ML.Load_Strat(loc=test_strat_loc,
              col_names='rel_family_id')

In [ ]:
#ML.strat.head()

Great, we now have our data, targets, covars, strat and exclusions loaded (Noting that the minimum requiriments for running an ML expiriment are just data or covars and targets, the rest being optional). The actual length of the script is also not as terrible as it seems, and once loading behavior is confirmed, verbose can even be turned off. To show this, we can re-load everything as above with verbose off. (Commented out, but you get the idea~)

In [ ]:
#ML = ABCD_ML.ABCD_ML(verbose = False) # Reloading the ML object itself to reset everything.

#ML.Load_Exclusions(loc=test_exclusion_loc)

#ML.Load_Name_Map(loc = test_mapping_loc,
#                 source_name_col = "NDAR name",
#                 target_name_col = "REDCap name/NDA alias")

#ML.Load_Data(loc=[test_data_loc1, test_data_loc2],
#             dataset_type='explorer',
#             filter_outlier_percent=.0005, 
#             winsorize_val=.01)

#ML.Load_Targets(loc=test_target_loc,
#                col_name='demo_sex_v2',
#                data_type='b')

#ML.Load_Covars(loc=test_target_loc,
#               col_names = 'demo_ed_v2',
#               data_types = 'ordinal',
#               standardize = False,
#               normalize = True)

#ML.Load_Strat(loc=test_strat_loc,
#              col_names='rel_family_id')

#ML.verbose = True

# Validation Phase

Lets move onto defining our validation stratagy (which is again optional, but as stated before for this example we are going to preserve family ids within the same folds), if no explicit validation strategy is defined, then random splits will be used.

In [ ]:
#help(ML.Define_Validation_Strategy)

So for group preserving behavior, as we are interested in keep families within the same folds, we supply an argument for groups. Specifically, we use the name of the column loaded within self.strat

In [ ]:
ML.Define_Validation_Strategy(groups='rel_family_id')

Lastly before we get to modelling, we want to define a global train-test split, so that we can perform model exploration, and parameter tuning ect... on a training set, and leave a left-out testing set to eventually test with out final selected model.

In [ ]:
#help(ML.Train_Test_Split)

In [ ]:
ML.Train_Test_Split(test_size=.25, #Let be somewhat conservative, and use a size of .25
                    random_state=1)

Great - and because we set the validation stratagy to preserve family structure within the folds, we know that no family id is in both the train and test set - for the paranoid we can make sure of this.

In [ ]:
#train_ids = set(ML.strat['rel_family_id'].loc[ML.train_subjects])
#test_ids = set(ML.strat['rel_family_id'].loc[ML.test_subjects])

#print('Unique family ids in train: ', len(train_ids))
#print('Unique family ids in test: ', len(test_ids))
#print('Overlap : ', len(train_ids.intersection(test_ids)))

# Modeling Phase

Now we can proceed to modeling.

The main function we use here is Evaluate, we can look at its docstring, but from a very high level, this is the function we use to test different expirimental setups.

In [ ]:
#help(ML.Evaluate)

Lets check what different model types we have avaliable for binary first.

In [ ]:
#ML.Show_Model_Types(problem_type='binary')

We can also look at different metrics avaliable for binary classification, and different data scalers.

In [ ]:
#ML.Show_Metrics(problem_type='binary')

In [ ]:
#ML.Show_Data_Scalers(show_scaler_help=False, show_default_params=True)

We can also set some default ML params for some of the settings that we will be keeping the same.

In [ ]:
#help(ML.Set_Default_ML_Params)

The parameters are mostly the same for setting default params as they are passed to Evaluate or Test. Importantly, by defining defaults, we define the value to be used if no value is passed to a given argument in Evaluate or Test.
Lets set some.

In [ ]:
ML.Set_Default_ML_Params(problem_type = 'binary', #This will be staying the same
                         metric = ['macro roc auc', 'f1', 'accuracy'], #Lets set a few
                         data_scaler = 'standard', #Standard scaling is fine, but this can also be a list
                         n_splits = 2, #For the sake of quick run-time and this example 2 splits, no repeats
                         n_repeats = 1,
                         int_cv = 3,
                         class_weight = 'balanced', 
                         n_jobs = 8,
                         n_iter = 8,
                         random_state = 1)

Now that we have some default ML params set, all we need to do to run an evaluation is simply:

In [ ]:
raw_scores = ML.Evaluate(model_type='rf',
                         sampler='smote',
                         ensemble_type = 'basic ensemble',
                         ensemble_split = .2,
                         n_repeats=1,
                         calc_shap_feature_importances = True,
                         calc_base_feature_importances = True)

In [ ]:
ML.Plot_Base_Feat_Importances(top_n = 10)

In [ ]:
ML.Plot_Shap_Feat_Importances()

In [ ]:
ML.Plot_Shap_Summary()

In [ ]:
vcbcvb

Lets try with multiple data scalers now, just to show off that functionality, and still just the logistic regression.

In [ ]:
raw_scores = ML.Evaluate(model_type='logistic',
             data_scaler = ['robust', 'minmax'])

The robust scaler and minmax scaler seem to help a bit, so hey lets set that as our new default... just for fun.

Note when setting a new param, you do not need to repass in everything you are not changing!

In [ ]:
ML.Set_Default_ML_Params(data_scaler = ['robust', 'minmax'])

# Let make just take a look at the new default params, to make sure it is as expected
ML.default_ML_params

We can run a more complicated classifier, in particular, let's
try the light gradient boosting machine (LGBM) classifier, with a random search for parameters.
Note: The ' rs' at the end of the str model indicator is a special key for selecting the random parameter search option.

In [ ]:
raw_scores = ML.Evaluate(model_type='lgbm classifier',
                         search_type='random',
                         model_type_param_ind=1,
                         n_iter=2)

Interesting, the basic logistic regression seems to perform better then the more complex gradient boosting. One thing to note though is that we only tested 8 random sets of hyperparamers, what if we tried exploring a larger number? (This might take a while, especially if you are not running this on a powerful computer).

In [ ]:
raw_scores = ML.Evaluate(model_type='lgbm classifier rs',
                         n_iter=100)

That is still not all that big a boost from just a logistic regression.
We can still explore some other linear classifiers though.

In [ ]:
raw_scores = ML.Evaluate(model_type='ridge logistic rs')

In [ ]:
raw_scores = ML.Evaluate(model_type='knn classifier gs')

In [ ]:
raw_scores = ML.Evaluate(model_type='gaussian nb')

In [ ]:
#This is a gaussian process classifier
raw_scores = ML.Evaluate(model_type='gp')

In [ ]:
raw_scores = ML.Evaluate(model_type='elastic net logistic rs')

You could easily continue with the model exploration portion, but for this example, lets say we have explored enough.

# Testing Phase

In [ ]:
ML.Test(model_type ='ridge logistic rs')